In [9]:
import os
from dotenv import load_dotenv
import requests
from pytz import timezone
import datetime
import pandas as pd

load_dotenv()
api_key = os.getenv("API_KEY")


In [3]:
api_url = "https://freeserv.dukascopy.com/2.0/?path=api/instrumentList"

response = requests.get(api_url, params={'key': api_key})

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Access the response data
    data = response.json()
    print(data)
else:
    print(f"Request failed with status code: {response.status_code}")

[{'id': '2000', 'name': 'AUD/JPY', 'pipValue': 0.01}, {'id': '2002', 'name': 'AUD/USD', 'pipValue': 0.0001}, {'id': '2004', 'name': 'CAD/JPY', 'pipValue': 0.01}, {'id': '2006', 'name': 'CHF/JPY', 'pipValue': 0.01}, {'id': '2008', 'name': 'EUR/CHF', 'pipValue': 0.0001}, {'id': '2010', 'name': 'EUR/DKK', 'pipValue': 0.0001}, {'id': '2012', 'name': 'EUR/GBP', 'pipValue': 0.0001}, {'id': '2014', 'name': 'EUR/JPY', 'pipValue': 0.01}, {'id': '2016', 'name': 'EUR/NOK', 'pipValue': 0.0001}, {'id': '2018', 'name': 'EUR/SEK', 'pipValue': 0.0001}, {'id': '2020', 'name': 'EUR/USD', 'pipValue': 0.0001}, {'id': '2022', 'name': 'GBP/CHF', 'pipValue': 0.0001}, {'id': '2024', 'name': 'GBP/JPY', 'pipValue': 0.01}, {'id': '2026', 'name': 'GBP/USD', 'pipValue': 0.0001}, {'id': '2028', 'name': 'USD/CAD', 'pipValue': 0.0001}, {'id': '2030', 'name': 'USD/CHF', 'pipValue': 0.0001}, {'id': '2032', 'name': 'USD/JPY', 'pipValue': 0.01}, {'id': '2034', 'name': 'NZD/USD', 'pipValue': 0.0001}, {'id': '2036', 'name'

In [4]:
api_url = "https://freeserv.dukascopy.com/2.0/?path=api/historicalPrices"

instrument = "2024"
timeFrame = "1min"


dt = datetime.datetime(2010, 1, 1, 0, 0, 0)
start_timestamp = int(dt.timestamp()*1000)

current_datetime = datetime.datetime.now()
end_timestamp = int(current_datetime.timestamp()*1000)

candles_per_request = 5000       # Number of candles to retrieve per request

# Calculate the number of requests needed based on the total number of candles
total_candles = (end_timestamp - start_timestamp) // (60 * 1000)  # Convert milliseconds to minutes
total_requests = (total_candles // candles_per_request) + 1

print(total_requests)


1407


In [5]:
# Initialize an empty list to store the retrieved data
bid_data = []

# Loop through each request
for request_number in range(total_requests):
    print(request_number)
    # Calculate the start and end timestamps for each request
    request_start = start_timestamp + (request_number * candles_per_request * 60 * 1000)  # Convert minutes to milliseconds
    request_end = request_start + (candles_per_request * 60 * 1000) - 1
    
    response = requests.get(api_url, params={'key': api_key,
                                             'instrument': instrument,
                                             'timeFrame': timeFrame,
                                             'count': 5000,
                                             'start': request_start,
                                             'end': request_end,
                                             'offerSide': 'B'})
    
    # Parse the response and extract the candle data
    if response.status_code == 200:
        response_data = response.json()
        candles = response_data['candles']
        
        # Append the retrieved candles to the historical_data list
        bid_data.extend(candles)
    else:
        print(f"Error in API request {request_number + 1}: {response.status_code}")
        break

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [26]:
# Initialize an empty list to store the retrieved data
ask_data = []

# Loop through each request
for request_number in range(total_requests):
    print(request_number)
    # Calculate the start and end timestamps for each request
    request_start = start_timestamp + (request_number * candles_per_request * 60 * 1000)  # Convert minutes to milliseconds
    request_end = request_start + (candles_per_request * 60 * 1000) - 1
    
    response = requests.get(api_url, params={'key': api_key,
                                             'instrument': instrument,
                                             'timeFrame': timeFrame,
                                             'count': 5000,
                                             'start': request_start,
                                             'end': request_end,
                                             'offerSide': 'A'})
    
    # Parse the response and extract the candle data
    if response.status_code == 200:
        response_data = response.json()
        candles = response_data['candles']
        
        # Append the retrieved candles to the historical_data list
        ask_data.extend(candles)
    else:
        print(f"Error in API request {request_number + 1}: {response.status_code}")
        break

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [37]:
df = bid_df.join(ask_df)

In [38]:
df

,timestamp,bid_open,bid_high,bid_low,bid_close,bid_volume,ask_open,ask_high,ask_low,ask_close,ask_volume
timestamp,,,,,,,,,,,
2012-01-11 01:37:00,1326245820000,118.906,118.916,118.905,118.916,51.75,118.930,118.933,118.924,118.931,49.50
2012-01-11 01:38:00,1326245880000,118.916,118.922,118.908,118.908,51.64,118.931,118.940,118.931,118.931,40.52
2012-01-11 01:39:00,1326245940000,118.908,118.908,118.904,118.904,34.51,118.932,118.933,118.928,118.932,32.26
2012-01-11 01:40:00,1326246000000,118.904,118.913,118.904,118.912,64.04,118.931,118.939,118.928,118.933,63.46
2012-01-11 01:41:00,1326246060000,118.914,118.918,118.914,118.918,106.50,118.936,118.942,118.936,118.942,95.27
...,...,...,...,...,...,...,...,...,...,...,...
2023-05-16 19:35:00,1684265700000,170.183,170.188,170.160,170.184,88.14,170.204,170.206,170.179,170.202,84.60
2023-05-16 19:36:00,1684265760000,170.183,170.187,170.168,170.178,58.80,170.201,170.203,170.186,170.197,68.08
2023-05-16 19:37:00,1684265820000,170.178,170.178,170.152,170.153,43.05,170.196,170.196,170.169,170.172,42.60


In [59]:
years = df.index.year.unique()
months = range(1,13)
current_year = datetime.datetime.now().year
current_month = datetime.datetime.now().month

for year in years:
    for month in months:
        if (current_year > year) | (current_month >= month):
            data_filter = df[(df.index.year == year) & (df.index.month == month)]
            file_loc = 's3://fx-data-raw/gbpjpy/oneminute/'+str(year)+'_'+str(month).zfill(2)+'.parquet'
            print(file_loc)
            data_filter.to_parquet(file_loc)

s3://fx-data-raw/gbpjpy/oneminute/2012_01.parquet
s3://fx-data-raw/gbpjpy/oneminute/2012_02.parquet
s3://fx-data-raw/gbpjpy/oneminute/2012_03.parquet
s3://fx-data-raw/gbpjpy/oneminute/2012_04.parquet
s3://fx-data-raw/gbpjpy/oneminute/2012_05.parquet
s3://fx-data-raw/gbpjpy/oneminute/2012_06.parquet
s3://fx-data-raw/gbpjpy/oneminute/2012_07.parquet
s3://fx-data-raw/gbpjpy/oneminute/2012_08.parquet
s3://fx-data-raw/gbpjpy/oneminute/2012_09.parquet
s3://fx-data-raw/gbpjpy/oneminute/2012_10.parquet
s3://fx-data-raw/gbpjpy/oneminute/2012_11.parquet
s3://fx-data-raw/gbpjpy/oneminute/2012_12.parquet
s3://fx-data-raw/gbpjpy/oneminute/2013_01.parquet
s3://fx-data-raw/gbpjpy/oneminute/2013_02.parquet
s3://fx-data-raw/gbpjpy/oneminute/2013_03.parquet
s3://fx-data-raw/gbpjpy/oneminute/2013_04.parquet
s3://fx-data-raw/gbpjpy/oneminute/2013_05.parquet
s3://fx-data-raw/gbpjpy/oneminute/2013_06.parquet
s3://fx-data-raw/gbpjpy/oneminute/2013_07.parquet
s3://fx-data-raw/gbpjpy/oneminute/2013_08.parquet


In [56]:
data_filter

,timestamp,bid_open,bid_high,bid_low,bid_close,bid_volume,ask_open,ask_high,ask_low,ask_close,ask_volume
timestamp,,,,,,,,,,,
2014-09-01 00:00:00,1409526000000,172.731,172.740,172.731,172.740,18.63,172.753,172.753,172.751,172.752,21.52
2014-09-01 00:01:00,1409526060000,172.740,172.740,172.737,172.737,24.00,172.757,172.760,172.749,172.750,36.72
2014-09-01 00:02:00,1409526120000,172.737,172.738,172.733,172.735,30.37,172.750,172.755,172.748,172.752,40.53
2014-09-01 00:03:00,1409526180000,172.735,172.739,172.731,172.733,25.49,172.754,172.754,172.748,172.749,37.50
2014-09-01 00:04:00,1409526240000,172.733,172.740,172.731,172.734,41.82,172.748,172.755,172.748,172.752,53.83
...,...,...,...,...,...,...,...,...,...,...,...
2014-09-30 23:55:00,1412117700000,177.746,177.746,177.738,177.738,10.40,177.767,177.767,177.759,177.759,8.24
2014-09-30 23:56:00,1412117760000,177.738,177.738,177.736,177.736,9.80,177.759,177.759,177.758,177.758,5.00
2014-09-30 23:57:00,1412117820000,177.736,177.736,177.732,177.736,6.13,177.758,177.758,177.755,177.758,5.82


In [43]:
data['candles']

[{'timestamp': 1684207140000,
  'bid_open': 170.368,
  'bid_high': 170.378,
  'bid_low': 170.361,
  'bid_close': 170.372,
  'bid_volume': 91.21},
 {'timestamp': 1684207200000,
  'bid_open': 170.373,
  'bid_high': 170.375,
  'bid_low': 170.363,
  'bid_close': 170.368,
  'bid_volume': 28.92},
 {'timestamp': 1684207260000,
  'bid_open': 170.367,
  'bid_high': 170.367,
  'bid_low': 170.358,
  'bid_close': 170.358,
  'bid_volume': 13.24},
 {'timestamp': 1684207320000,
  'bid_open': 170.358,
  'bid_high': 170.365,
  'bid_low': 170.35,
  'bid_close': 170.364,
  'bid_volume': 41.51},
 {'timestamp': 1684207380000,
  'bid_open': 170.363,
  'bid_high': 170.388,
  'bid_low': 170.363,
  'bid_close': 170.385,
  'bid_volume': 24},
 {'timestamp': 1684207440000,
  'bid_open': 170.387,
  'bid_high': 170.391,
  'bid_low': 170.384,
  'bid_close': 170.385,
  'bid_volume': 21.69},
 {'timestamp': 1684207500000,
  'bid_open': 170.382,
  'bid_high': 170.391,
  'bid_low': 170.377,
  'bid_close': 170.39,
  'bid_

TypeError: list indices must be integers or slices, not str

In [41]:
5000/60/24

3.472222222222222

In [45]:

start_timestamp = 1678924800  # Example start timestamp
end_timestamp = 1681599600   # Example end timestamp
candles_per_request = 5000       # Number of candles to retrieve per request

# Initialize an empty list to store the retrieved data
historical_data = []

# Calculate the number of requests needed based on the total number of candles
total_candles = end_timestamp - start_timestamp
total_requests = (total_candles // candles_per_request) + 1

# Loop through each request
for request_number in range(total_requests):
    # Calculate the start and end timestamps for each request
    request_start = start_timestamp + (request_number * candles_per_request)
    request_end = request_start + candles_per_request - 1
    
    response = requests.get(api_url, params={'key': api_key,
                                             'instrument': instrument,
                                             'timeFrame': timeFrame,
                                             'start': request_start,
                                             'end': request_end,})
    
    # Parse the response and extract the candle data
    if response.status_code == 200:
        response_data = response.json()
        candles = response_data['candles']
        
        # Append the retrieved candles to the historical_data list
        historical_data.extend(candles)
    else:
        print(f"Error in API request {request_number + 1}: {response.status_code}")
        break

# Print the condensed historical data
for candle in historical_data:
    print(candle)

Error in API request 1: 204


In [114]:
api_url = "https://freeserv.dukascopy.com/2.0/?path=api/historicalPrices"

response = requests.get(api_url, params={'key': api_key,
                                            'instrument': instrument,
                                            'timeFrame': timeFrame,
                                            'count': 5000,
                                            'start': start_timestamp})


In [127]:
dt = datetime.datetime(2010, 1, 1, 0, 0, 0)
start_timestamp = int(dt.timestamp()*1000)

In [128]:

# Parse the response and extract the candle data
if response.status_code == 200:
    response_data = response.json()
    candles = response_data['candles']
else:
    print(f"Error in API request {request_number + 1}: {response.status_code}")


In [129]:
candles

[{'timestamp': 1683768180000,
  'bid_open': 169.42,
  'bid_high': 169.443,
  'bid_low': 169.412,
  'bid_close': 169.436,
  'bid_volume': 67.9},
 {'timestamp': 1683768240000,
  'bid_open': 169.436,
  'bid_high': 169.443,
  'bid_low': 169.409,
  'bid_close': 169.419,
  'bid_volume': 115.12},
 {'timestamp': 1683768300000,
  'bid_open': 169.419,
  'bid_high': 169.419,
  'bid_low': 169.379,
  'bid_close': 169.384,
  'bid_volume': 129.74},
 {'timestamp': 1683768360000,
  'bid_open': 169.386,
  'bid_high': 169.397,
  'bid_low': 169.38,
  'bid_close': 169.39,
  'bid_volume': 57.36},
 {'timestamp': 1683768420000,
  'bid_open': 169.39,
  'bid_high': 169.414,
  'bid_low': 169.386,
  'bid_close': 169.407,
  'bid_volume': 61.99},
 {'timestamp': 1683768480000,
  'bid_open': 169.408,
  'bid_high': 169.439,
  'bid_low': 169.404,
  'bid_close': 169.439,
  'bid_volume': 39.72},
 {'timestamp': 1683768540000,
  'bid_open': 169.442,
  'bid_high': 169.443,
  'bid_low': 169.428,
  'bid_close': 169.437,
  'bi